# [실습] OpenAI API를 이용하여 뉴스 기사 정리 봇 만들기

OpenAI Chat API를 이용해 뉴스 검색 결과를 정리하는 프로그램을 만들어 보겠습니다.


네이버의 뉴스 검색 API를 사용하겠습니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)

In [38]:
# Agent처럼 별도 다른 버전으로 맞춘 패키지를 별도 가상 환경에서 동작시키기 위한 예시 (다음 코드는 최신 버전에서도 정상 동작함)
!pip install openai>=1.43.0 tiktoken==0.7.0 python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# .env 파일 불러오기
from dotenv import load_dotenv
load_dotenv()

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

OPENAI_API_KEY가 정상적으로 설정되어 있습니다.


## 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기

네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성하겠습니다.   
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 전달하는 함수를 요청합니다.

**네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 올라갑니다.**

In [10]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 아래 포맷의 문자열로 변환하는 함수 get_news(query) 를 만들어 주세요.
---
제목: (뉴스 제목)
URL: (뉴스 링크- 있으면)
내용: (뉴스 내용)
---

결과는 result에 저장하고, 출력하세요.
---




# 검색어
query = '생성형 AI'


# 검색 결과를 return하는 함수
def get_news(query):


'''



In [11]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{'role':'system', 'content':'당신은 파이썬 코딩의 전문가입니다. 설명 없이 코드만 출력하세요.'},
          {'role':'user', 'content': prompt}],

    temperature =  0,
    max_tokens = 2000

)
print(response.choices[0].message.content)

```python
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': 'YOUR_CLIENT_ID',
        'X-Naver-Client-Secret': 'YOUR_CLIENT_SECRET'
    }
    
    response = requests.get(url, headers=headers)
    news_data = response.json()
    
    result = ""
    for item in news_data['items']:
        title = item['title']
        link = item['link']
        description = item['description']
        
        result += f"제목: {title}\n"
        result += f"URL: {link}\n"
        result += f"내용: {description}\n"
        result += "---\n"
    
    return result

# 검색어
query = '생성형 AI'
result = get_news(query)
print(result)
```


In [12]:
# VS CODE에서 위 결과가 truncated 되는 경우 scrollable element를 클릭하여 풀 코드 확인

아래의 API 키를 사용합니다.
(생성된 코드에 값을 추가하세요)

In [13]:
# 네이버 API Key : 개발자 센터에서 발급 가능
# 본인 API Key를 발급 받아도 되고, 실습 시에는 다음 정보를 사용하셔도 됩니다.(SKALA 교수진용)

headers = {
    'X-Naver-Client-Id': 'gbqzUVViEiF6WXhuq3gZ',
    'X-Naver-Client-Secret': 'y0YXaa5unU'
}

아래에 전체 코드를 넣고 실행합니다.

In [14]:
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': 'gbqzUVViEiF6WXhuq3gZ',
        'X-Naver-Client-Secret': 'y0YXaa5unU'
    }
    
    response = requests.get(url, headers=headers)
    news_items = response.json().get('items', [])
    
    result = ""
    for item in news_items:
        title = item['title']
        link = item['link']
        description = item['description']
        
        result += f"제목: {title}\n"
        result += f"URL: {link}\n"
        result += f"내용: {description}\n"
        result += "---\n"
    
    print(result)
    return result

# 검색어
query = '생성형 AI'
get_news(query)


제목: [<b>AI</b>돋보기] <b>AI</b> 시대, 당신의 직업은 안전한가
URL: https://n.news.naver.com/mnews/article/001/0015569039?sid=105
내용: 이미 <b>생성형 AI</b>는 고객 응대나 마케팅 콘텐츠 제작 등 사무직 영역 일부를 대체하고 있으며 그 영향력은 이제 IT 인프라와 엔지니어링, 나아가 정책 관리 부문으로까지 확산하는 추세다. 이제는 &quot;AI가 일하고, 사람은... 
---
제목: 올트먼 &quot;박사로 진화&quot; 자신하더니…망신살 뻗친 오픈<b>AI</b> '야심작'
URL: https://n.news.naver.com/mnews/article/015/0005171342?sid=105
내용: CNN은 GPT-5에 대한 사용자 실망을 오픈AI의 과대광고에 따른 필연적 결과로 분석했다. 챗GPT는 현재 주간 이용자가 약 7억명으로 세계에서 가장 많이 사용되는 <b>생성형 AI</b>다.
---
제목: 8.15㎞ 기부런, <b>AI</b>로 유관순 부활... MZ세대가 광복절 기념하는 법
URL: https://n.news.naver.com/mnews/article/023/0003923344?sid=102
내용: 보기만 해도 시원해 보이는 이 영상은 유관순 열사가 서대문형무소에 수감됐을 때 남겨진 흑백사진에 <b>생성형 AI</b>(인공지능)로 현대적 의복과 배경, 색채를 더해 새롭게 만든 것이다. 영상의 제목은 ‘독립운동가분들께... 
---
제목: SK하이닉스, 자체 업무용 <b>생성형 AI</b> 플랫폼 '가이아' 공개
URL: https://n.news.naver.com/mnews/article/031/0000957229?sid=105
내용: SK하이닉스가 14일 자사 뉴스룸을 통해 반도체 업무에 특화된 <b>생성형 AI</b> 플랫폼 '가이아'를 공개했다. 회사는 가이아(GaiA, Generative AI Assistant)를 중심으로 에이전틱 AI와 에이전트 오케

"제목: [<b>AI</b>돋보기] <b>AI</b> 시대, 당신의 직업은 안전한가\nURL: https://n.news.naver.com/mnews/article/001/0015569039?sid=105\n내용: 이미 <b>생성형 AI</b>는 고객 응대나 마케팅 콘텐츠 제작 등 사무직 영역 일부를 대체하고 있으며 그 영향력은 이제 IT 인프라와 엔지니어링, 나아가 정책 관리 부문으로까지 확산하는 추세다. 이제는 &quot;AI가 일하고, 사람은... \n---\n제목: 올트먼 &quot;박사로 진화&quot; 자신하더니…망신살 뻗친 오픈<b>AI</b> '야심작'\nURL: https://n.news.naver.com/mnews/article/015/0005171342?sid=105\n내용: CNN은 GPT-5에 대한 사용자 실망을 오픈AI의 과대광고에 따른 필연적 결과로 분석했다. 챗GPT는 현재 주간 이용자가 약 7억명으로 세계에서 가장 많이 사용되는 <b>생성형 AI</b>다.\n---\n제목: 8.15㎞ 기부런, <b>AI</b>로 유관순 부활... MZ세대가 광복절 기념하는 법\nURL: https://n.news.naver.com/mnews/article/023/0003923344?sid=102\n내용: 보기만 해도 시원해 보이는 이 영상은 유관순 열사가 서대문형무소에 수감됐을 때 남겨진 흑백사진에 <b>생성형 AI</b>(인공지능)로 현대적 의복과 배경, 색채를 더해 새롭게 만든 것이다. 영상의 제목은 ‘독립운동가분들께... \n---\n제목: SK하이닉스, 자체 업무용 <b>생성형 AI</b> 플랫폼 '가이아' 공개\nURL: https://n.news.naver.com/mnews/article/031/0000957229?sid=105\n내용: SK하이닉스가 14일 자사 뉴스룸을 통해 반도체 업무에 특화된 <b>생성형 AI</b> 플랫폼 '가이아'를 공개했다. 회사는 가이아(GaiA, Generative AI Assistant)를 중심으로 에

아래는 예시 코드입니다.

## 2. OpenAI chat을 이용하여 요약하기

키워드에 대한 뉴스 검색 결과를 요약합니다.   
위 결과인 `result` 데이터를 입력으로 넣은 뒤, 해당 결과에 대한 요약을 요청합니다.

In [15]:
def news_bot(messages):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages,
        temperature = 0.1,
        max_tokens= 2048,
    )

    return response.choices[0].message.content
    # 간편한 출력을 위해 content만 추출

챗봇에 전달할 메시지는 두 개로 나눠집니다.   
위에서 얻은 get_html(query)와, 사용자가 제시할 명령입니다.   

In [16]:
query = '삼성 라이온즈'

result = get_news(query)

response = news_bot([
    {
        # f-string: f"""{변수명}"""
        "role": "system",
        "content": f"""{query}에 대한 뉴스 검색 결과가 주어집니다.
{query}에 대한 뉴스를 요약하세요.
관련 링크를 추가하세요.
---
"""
    },
    {
        "role": "user",
        "content": result
    }
])

print(response)

제목: '최원태 쾌투·구자욱 쐐기포' <b>삼성</b>, 5연패 뒤 2연승…롯데 8연패 '늪'
URL: https://m.sports.naver.com/kbaseball/article/003/0013425155
내용: 프로야구 <b>삼성 라이온즈</b>가 5연패 뒤 2연승을 달리며 반등에 성공했다. 삼성은 16일 사직구장에서 벌어진 2025 신한 쏠뱅크 KBO리그 롯데와의 경기에서 4-1로 승리했다. 전날 롯데를 10-4로 물리치고 5연패를 끊은 삼성은... 
---
제목: <b>삼성</b>, '단두대 매치'서 5연패 탈출…롯데 7연패 악몽
URL: https://m.sports.naver.com/kbaseball/article/421/0008430719
내용: 프로야구 <b>삼성 라이온즈</b>가 롯데 자이언츠를 완파하고 5연패에서 벗어났다. 무기력한 모습을 보인 롯데는 1년 4개월 만에 7연패 수모를 당했다. 삼성은 15일 부산 사직구장에서 열린 롯데와 2025 신한 SOL뱅크 KBO리그... 
---
제목: [사진] 롯데전 2연승 챙긴 <b>삼성</b> 선수들
URL: https://m.sports.naver.com/kbaseball/article/109/0005372217
내용: 16일 부산 사직야구장에서 2025 신한 SOL 뱅크 KBO 리그 롯데 자이언츠와 <b>삼성 라이온즈</b>의 경기가 열렸다. 홈팀 롯데는 박세웅이, 방문팀 삼성은 최원태가 선발 출전했다. <b>삼성 라이온즈</b> 선수들이 롯데... 
---
제목: '70억 FA' 부활 <b>삼성</b>, 최원태+구자욱 앞세워 2연승…롯데 8연패 수렁 [사...
URL: https://m.sports.naver.com/kbaseball/article/311/0001905414
내용: 박진만 감독이 이끄는 <b>삼성 라이온즈</b>가 5연패 탈출 후 곧바로 연승을 내달렸다. 베테랑 우완 최원태가 '부활투'를 펼치면서 주말 3연전 위닝 시리즈를 확보했다. 삼성은 16일 부산 사직야구장에서 열린 

In [18]:
# 토큰 수 체크
import tiktoken

tokenizer  = tiktoken.encoding_for_model('gpt-4o')

print("문자 수:", len(result))
print("토큰 수:", len(tokenizer.encode(result)))
# print(result) #result


문자 수: 7719
토큰 수: 4783


다양한 명령을 넣어서 형식과 스타일을 바꿀 수 있습니다.

In [19]:
query = '삼성 라이온즈'

result  = get_news(query)

result = news_bot([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": f"""
---
위 전체 내용을 종합하여 {query}의 동향과 미래에 대해 논쟁하는 두 사람의 대화 내용을 만들어줘.
한 명은 전문적인 말투, 한 명은 어리숙한 말투를 사용하고, 형식은 아래와 같아.
---
A:(A의 대화)
B:(B의 대화)
        """
    }
])

print(result)

제목: '최원태 쾌투·구자욱 쐐기포' <b>삼성</b>, 5연패 뒤 2연승…롯데 8연패 '늪'
URL: https://m.sports.naver.com/kbaseball/article/003/0013425155
내용: 프로야구 <b>삼성 라이온즈</b>가 5연패 뒤 2연승을 달리며 반등에 성공했다. 삼성은 16일 사직구장에서 벌어진 2025 신한 쏠뱅크 KBO리그 롯데와의 경기에서 4-1로 승리했다. 전날 롯데를 10-4로 물리치고 5연패를 끊은 삼성은... 
---
제목: <b>삼성</b>, '단두대 매치'서 5연패 탈출…롯데 7연패 악몽
URL: https://m.sports.naver.com/kbaseball/article/421/0008430719
내용: 프로야구 <b>삼성 라이온즈</b>가 롯데 자이언츠를 완파하고 5연패에서 벗어났다. 무기력한 모습을 보인 롯데는 1년 4개월 만에 7연패 수모를 당했다. 삼성은 15일 부산 사직구장에서 열린 롯데와 2025 신한 SOL뱅크 KBO리그... 
---
제목: [사진] 롯데전 2연승 챙긴 <b>삼성</b> 선수들
URL: https://m.sports.naver.com/kbaseball/article/109/0005372217
내용: 16일 부산 사직야구장에서 2025 신한 SOL 뱅크 KBO 리그 롯데 자이언츠와 <b>삼성 라이온즈</b>의 경기가 열렸다. 홈팀 롯데는 박세웅이, 방문팀 삼성은 최원태가 선발 출전했다. <b>삼성 라이온즈</b> 선수들이 롯데... 
---
제목: '70억 FA' 부활 <b>삼성</b>, 최원태+구자욱 앞세워 2연승…롯데 8연패 수렁 [사...
URL: https://m.sports.naver.com/kbaseball/article/311/0001905414
내용: 박진만 감독이 이끄는 <b>삼성 라이온즈</b>가 5연패 탈출 후 곧바로 연승을 내달렸다. 베테랑 우완 최원태가 '부활투'를 펼치면서 주말 3연전 위닝 시리즈를 확보했다. 삼성은 16일 부산 사직야구장에서 열린 

## 참고) 스트리밍
ChatGPT처럼 글자가 한 글자씩 순서대로 쭉 출력되도록 하면 어떨까요?   
이와 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있습니다.

In [11]:
def news_bot_stream(messages):

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens = 2048,
        stream=True # 스트리밍
    )

    return response
    # content가 아닌 전체를 return함

In [12]:
query = '거대 언어 모델'
result = get_news(query)

result = news_bot_stream([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": """
위 전체 내용을 종합하여 뉴스 리포팅을 해줘.
관련 링크를 추가해줘
---
'제목':
'본문':
        """
    }
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

제목: 수백만 AI ‘초<b>거대</b> 집단지성’…인간 통제될까? [AI 엑스파일]
URL: https://n.news.naver.com/mnews/article/015/0005171395?sid=105
내용: 지금의 대형<b>언어모델</b>(LLM)은 인간보다 훨씬 더 많은 내용을 접해서 초인적인 지식을 갖게 됐습니다. 앞으로... 모든 경험과 지식을 실시간으로 공유하는 '초<b>거대</b> 집단 지성'이다. 수백만 개의 AI가 각자의 영역에서 24시간... 
---
제목: “소버린 AI 원조, 독자LLM <b>모델</b> 가진 네이버가 국가대표 AI”
URL: https://n.news.naver.com/mnews/article/023/0003923143?sid=105
내용: 이미 자체 <b>거대언어모델</b>(LLM) ‘하이퍼클로바X’를 내놓았고, 데이터센터, 클라우드, 모델, 서비스까지 풀스택(Full-Stack·인프라, 하드웨어, 소프트웨어 등 기술을 패키지로 제공) 역량을 갖춘 컨소시엄이기도 하다. 성... 
---
제목: [K-AI 도전]④네이버 &quot;대한민국 구석구석 가장 잘 아는 AI <b>모델</b>…수출도...
URL: https://n.news.naver.com/mnews/article/277/0005637163?sid=105
내용: 네이버클라우드 컨소시엄은 과학기술정보통신부가 국가대표 AI를 뽑는 '독자 AI 파운데이션 <b>모델</b>' 사업에... 옴니 모달리티는 <b>언어</b>뿐만 아니라 이미지, 음성, 영상 등 다양한 데이터를 동시에 이해하고 처리한다. 인간의... 
---
제목: [국대 AI人]&quot;누구나 활용 가능한 최고 성능 K-AI <b>모델</b> 선보일 것&quot;
URL: https://n.news.naver.com/mnews/article/008/0005235749?sid=101
내용: 네이버의 핵심 AI 개발자 출신들이 2020년 설립한 업스테이지는 <b>거대언어모델</b>(LLM) '솔라'를 개발

이와 같은 스트리밍은 LLM 어플리케이션의 마지막 출력에서 사용하면    
사용자 경험을 향상시킬 수 있습니다.

<br><br><br>
## Tool(과거 Function Call) : 커스텀 함수 사용하기
만약 query를 명시적으로 지정하지 않고, 사용자의 질문에 따라 관련 query를 추출하고,  
이를 통해 검색하게 할 수도 있을까요?    

`Tool Calling` 를 추가하면 이를 수행할 수 있습니다.    

Web_Search라는 이름의 툴을 만들고, 이를 `get_html`과 연결하겠습니다.

-------


In [13]:
from pydantic import BaseModel, Field

# 1. Pydantic 모델 정의
class WebSearchInput(BaseModel):
    """query를 이용하여 뉴스 검색"""
    query: str = Field(
        description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
"""
    )

# 2. Function tool 수동 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "query를 이용하여 뉴스 검색",
            "parameters": WebSearchInput.model_json_schema()  # Pydantic v2 대응
        }
    }
]

# 확인
from pprint import pprint
pprint(tools)


[{'function': {'description': 'query를 이용하여 뉴스 검색',
               'name': 'web_search',
               'parameters': {'description': 'query를 이용하여 뉴스 검색',
                              'properties': {'query': {'description': '검색 키워드\n'
                                                                      '규칙:\n'
                                                                      '1. 최대 '
                                                                      '2개 단어로 '
                                                                      '구성\n'
                                                                      '2. 불필요한 '
                                                                      '조사나 형용사 '
                                                                      '제외\n'
                                                                      '3. 핵심 '
                                                                      '명사만 포함\n'
                                                    

Tool 정보가 포함된 gpt는 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단합니다.

직접적으로 실행하는 것은 아니지만, 결과를 활용하면 해당 함수를 바로 사용할 수 있습니다.

In [20]:
from openai import OpenAI
from pydantic import BaseModel, Field

client = OpenAI()

def news_bot_v2(messages, stream=False, model='gpt-4o'):

    # 1. Pydantic 모델 정의
    class WebSearchInput(BaseModel):
        """query를 이용하여 뉴스 검색"""
        query: str = Field(description="""
검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
""")

    # 2. tool 수동 정의
    tools = [
        {
            "type": "function",
            "function": {
                "name": "web_search",
                "description": "query를 이용하여 뉴스 검색",
                "parameters": WebSearchInput.model_json_schema()
            }
        }
    ]

    # 3. chat completion 호출
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.1,
        max_tokens=1024,
        stream=stream
    )

    # 4. 결과 리턴
    return response if stream else response.choices[0].message


In [21]:
# tool 사용 필요 없음
result = news_bot_v2([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])

result

ChatCompletionMessage(content='안녕하세요! 날씨가 좋다니 기분이 좋으시겠어요. 오늘 하루도 즐겁게 보내세요! 혹시 도움이 필요하신 부분이 있으면 말씀해 주세요.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [22]:
# tool 사용 필요함

tool_call_result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    }
])

print(tool_call_result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_GGADRSr4LBl7ansgLjqAoKZY', function=Function(arguments='{"query":"영화 개봉"}', name='web_search'), type='function')])


Tool Call 메시지는 content 대신 tool_calls로 전달됩니다.

In [23]:
tool_call_result.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_GGADRSr4LBl7ansgLjqAoKZY', function=Function(arguments='{"query":"영화 개봉"}', name='web_search'), type='function')]

In [24]:
(tool_call_result.tool_calls[0].id,
tool_call_result.tool_calls[0].function.name,
tool_call_result.tool_calls[0].function.arguments)

('call_GGADRSr4LBl7ansgLjqAoKZY', 'web_search', '{"query":"영화 개봉"}')

LLM이 스스로 판단하여, 실행할 함수의 정보를 `tool_calls`에 돌려줍니다.

tool_calls의 구성 요소는 다음과 같습니다.
- id: tool call의 id로, 해당 id에 실행 결과를 연결할 수 있습니다.
- function : arguments와 name을 통해 실행할 툴의 이름과 매개변수를 전달합니다.
<br><br>


tool_calls의 결과를 임의로 포함하여 전달해 보겠습니다.   
`tool`타입의 메시지로 전달해야 합니다.

In [25]:
result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": """
청설, 보통의 가족, 아노라, 아마존 활명수, 베놈:라스트 댄스
""",
        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

result

ChatCompletionMessage(content='요즘 새로 개봉한 영화로는 "청설", "보통의 가족", "아노라", "아마존 활명수", "베놈: 라스트 댄스" 등이 있습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

이번에는 실제 함수를 실행하여 전달해 보겠습니다.

In [26]:
tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

('web_search', '{"query":"영화 개봉"}')

함수명과 함수에 전달할 매개변수값이 주어지면, 아래와 같이 함수를 불러올 수 있습니다.

In [27]:
# 문자열을 Dict로 바꾸기
import json

example = '{"query":"LLM"}'
example_dict = json.loads(example)
# json으로 변환

print(example_dict)
print(type(example_dict))



{'query': 'LLM'}
<class 'dict'>


In [28]:
# 함수이름 문자열을 함수로 바꾸기

available_functions = {'Web_Search': get_news}

available_functions['Web_Search']

<function __main__.get_news(query)>

In [29]:
# Dictionary를 함수의 매개변수로 전달하기

def f(a,b):
  return a + b

example2 = {'a': 1, 'b': 2}

f(**example2)


3

In [30]:
available_functions['Web_Search'](**example_dict)

제목: “소버린 AI 원조, 독자<b>LLM</b> 모델 가진 네이버가 국가대표 AI”
URL: https://n.news.naver.com/mnews/article/023/0003923143?sid=105
내용: 이미 자체 거대언어모델(<b>LLM</b>) ‘하이퍼클로바X’를 내놓았고, 데이터센터, 클라우드, 모델, 서비스까지... 거대언어모델(<b>LLM</b>)을 발표했고 소버린 AI라는 표현을 가장 먼저 사용한 기업도 네이버”라고 했다. 이번... 
---
제목: <b>LLM</b>의 시대, 인간의 시대
URL: https://www.lawtimes.co.kr/opinion/210528
내용: 메타발 인재 쓸어담기 열풍에 전 세계 AI 뒤처질까 불안 고조 <b>LLM</b> 기반 초지능 실현은 회의적 딥러닝 구조... 언어모델(<b>LLM</b>)에 천착한다. 그러나 나는 <b>LLM</b>이 초지능이 될 수 없다고 생각한다. 그리고 <b>LLM</b>을 개발하기 위해... 
---
제목: 한국형 경량 <b>LLM</b>, AI 생존 전략 실마리되나
URL: https://n.news.naver.com/mnews/article/030/0003341071?sid=105
내용: 대규모언어모델(<b>LLM</b>)이 한국의 글로벌 AI 생존 전략에서 새로운 선택지로 부상하고 있다. 최근 국내 한 스타트업이 공개한 31B(310억) 파라미터급 경량 <b>LLM</b>은 일부 주요 벤치마크 과제에서 GPT-4.1, 클로드 3.7 등 대형 모델을... 
---
제목: SK하이닉스에 'AI 홍반장' 떴다
URL: https://n.news.naver.com/mnews/article/009/0005541845?sid=101
내용: 현재까지 가이아를 기반으로 만들어진 생성형 AI 서비스는 '비즈(Biz) 특화' '엘엘엠 챗(<b>LLM</b> Chat)' '에이닷... <b>LLM</b> Chat은 사내 모든 직원이 이용할 수 있는 SK하이닉스 전용 챗GPT 서비스라고

"제목: “소버린 AI 원조, 독자<b>LLM</b> 모델 가진 네이버가 국가대표 AI”\nURL: https://n.news.naver.com/mnews/article/023/0003923143?sid=105\n내용: 이미 자체 거대언어모델(<b>LLM</b>) ‘하이퍼클로바X’를 내놓았고, 데이터센터, 클라우드, 모델, 서비스까지... 거대언어모델(<b>LLM</b>)을 발표했고 소버린 AI라는 표현을 가장 먼저 사용한 기업도 네이버”라고 했다. 이번... \n---\n제목: <b>LLM</b>의 시대, 인간의 시대\nURL: https://www.lawtimes.co.kr/opinion/210528\n내용: 메타발 인재 쓸어담기 열풍에 전 세계 AI 뒤처질까 불안 고조 <b>LLM</b> 기반 초지능 실현은 회의적 딥러닝 구조... 언어모델(<b>LLM</b>)에 천착한다. 그러나 나는 <b>LLM</b>이 초지능이 될 수 없다고 생각한다. 그리고 <b>LLM</b>을 개발하기 위해... \n---\n제목: 한국형 경량 <b>LLM</b>, AI 생존 전략 실마리되나\nURL: https://n.news.naver.com/mnews/article/030/0003341071?sid=105\n내용: 대규모언어모델(<b>LLM</b>)이 한국의 글로벌 AI 생존 전략에서 새로운 선택지로 부상하고 있다. 최근 국내 한 스타트업이 공개한 31B(310억) 파라미터급 경량 <b>LLM</b>은 일부 주요 벤치마크 과제에서 GPT-4.1, 클로드 3.7 등 대형 모델을... \n---\n제목: SK하이닉스에 'AI 홍반장' 떴다\nURL: https://n.news.naver.com/mnews/article/009/0005541845?sid=101\n내용: 현재까지 가이아를 기반으로 만들어진 생성형 AI 서비스는 '비즈(Biz) 특화' '엘엘엠 챗(<b>LLM</b> Chat)' '에이닷... <b>LLM</b> Chat은 사내 모든 직원이 이용할 수 있는 SK하이닉

위 과정을 조합하여 실행해 보겠습니다.

In [31]:
import json

# 1. 툴 함수 이름에 맞춰 딕셔너리 구성
available_functions = {'web_search': get_news}  # tool의 "name" 키와 일치해야 함

# 2. 툴 호출 결과로부터 이름과 인자 추출
tool_call = tool_call_result.tool_calls[0]
name = tool_call.function.name  # ex: "web_search"
arguments = json.loads(tool_call.function.arguments)  # 문자열 → dict

# 3. LLM에게 다시 응답 요청
result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 오늘 날짜는 24년 11월 18일입니다.
"""
    },
    tool_call_result,  # tool_calls 포함 assistant 응답
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": available_functions[name](**arguments)  # get_news(query="영화")
    }
])

# 4. 결과 출력
print(result.content)


제목: 李대통령 “저와 <b>영화</b>볼 분 모십니다…조진웅 배우 추천 ‘독립군’”
URL: https://n.news.naver.com/mnews/article/020/0003654578?sid=100
내용: 그는 13일 <b>개봉</b>한 다큐멘터리 <b>영화</b> ‘독립군: 끝나지 않은 전쟁’ 내레이터로도 참여했다. 조진웅은 지난해 윤석열 전 대통령 탄핵 촉구 집회에 응원 영상을 보내 “국민을 향해 극악무도하게도 비상계엄을 했다”고... 
---
제목: 李대통령, 내일 <b>영화</b> ‘독립군’ 본다…국민과 동반 관람
URL: https://n.news.naver.com/mnews/article/366/0001100695?sid=100
내용: <b>영화</b> ‘독립군: 끝나지 않은 전쟁‘은 독립운동가 홍범도 장군의 삶을 다룬 다큐멘터리 <b>영화</b>로, 지난 13일 <b>개봉</b>했다. 배우 조진웅은 <b>영화</b> 나레이션 작업에 참여했다. 조진웅은 전날 제80주년 광복절 경축식에서 국기에... 
---
제목: 조정석 ‘좀비딸’, 올 <b>개봉</b> <b>영화</b> 중 처음으로 관객 수 400만 돌파…북미...
URL: https://n.news.naver.com/mnews/article/056/0012010101?sid=103
내용: 이는 올해 국내에 <b>개봉</b>한 <b>영화</b> 중 브래드 피트 주연의 ‘F1: 더 무비’, 톰 크루즈 주연의 ‘미션 임파서블:파이널 레코닝’을 제치고 최초 400만 관객 돌파이자 최단기간 세운 최고 흥행 기록입니다. 또 ‘좀비딸’의... 
---
제목: 스크린 속 레시피로 만든 '치킨' 먹으면서 <b>영화</b> 본다고? [무비인사이드...
URL: https://n.news.naver.com/mnews/article/015/0005171478?sid=103
내용: 이 <b>영화</b>는 <b>개봉</b> 첫날 좌석점유율 61%를 기록하며, 단일관 상영임에도 평균 <b>영화</b> 흥행 기준(4

스트리밍 기능을 추가하고, 전체 과정을 함수로 자동화합니다.:

In [32]:
def news_qa(prompt):
    print('Prompt:', prompt)

    # Tool name과 실제 함수 연결
    available_functions = {'web_search': get_news}  # tool 정의의 name과 동일하게 유지

    # 1차 LLM 응답 받기
    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        }
    ])

    print('---')
    print('News_Bot: Call ', end='')

    # tool 호출이 존재할 경우
    if tool_call_result.tool_calls:
        tool_call = tool_call_result.tool_calls[0]
        name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)  # JSON 문자열 → dict

        print(name, arguments)

        # 2차 실제 tool 실행
        tool_result = available_functions[name](**arguments)

        print('---')
        print('News_Bot:', end='')

        # 3차: Tool 결과를 포함한 최종 답변 요청
        response = news_bot_v2(
            [
                {
                    "role": "system",
                    "content": "사용자의 질문과 이를 답변하기 위한 정보가 주어집니다. 이를 바탕으로 답변하세요."
                },
                {
                    "role": "user",
                    "content": prompt
                },
                tool_call_result,  # assistant message + tool_calls
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result
                }
            ],
            stream=True
        )

        for chunk in response:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                print(delta.content, end='')

    else:
        # 툴 사용 없이 바로 응답된 경우
        print('Nothing')
        print(tool_call_result.content)


# 예시 실행
prompt = """넷플릭스 신작 추천해줘."""
news_qa(prompt)


Prompt: 넷플릭스 신작 추천해줘.
---
News_Bot: Call web_search {'query': '넷플릭스 신작'}
제목: 다큐멘터리부터 SF 호러물까지⋯국내외 OTT <b>신작</b> 공개 [콘슐랭]
URL: https://n.news.naver.com/mnews/article/031/0000957584?sid=105
내용: 그날 그곳에 있었던 그들의 이야기 '나는 생존자다' <b>넷플릭스</b>는 15일 <b>신작</b>으로 '나는 생존자다'를 공개했다. 대한민국을 충격에 빠뜨린 네 개의 사건, 반복돼서는 안 될 이야기를 살아남은 사람들의 목소리로 기록한... 
---
제목: '나는 신이다' 후속작→'연애 실험', <b>넷플릭스 신작</b> '종합선물세트'
URL: https://sports.hankooki.com/news/articleView.html?idxno=6906955
내용: 8월 셋째 주, <b>넷플릭스</b>가 전 세계 시청자들의 시선을 사로잡을 <b>신작</b> 라인업을 공개했다. 참혹한 사건의 생존자 증언을 담은 다큐멘터리부터 숨 막히는 하룻밤 추격극, 얼굴 대신 마음으로만 이어지는 연애 실험, 인기... 
---
제목: [이번주 뭐보지] <b>넷플릭스</b> '나는 생존자다' 등 <b>신작</b> 공개
URL: https://www.newscj.com/news/articleView.html?idxno=3306067
내용: <b>넷플릭스</b>가 금주 <b>신작</b> 라인업을 15일 공개했다. '나는 생존자다'는 대한민국 사회를 뒤흔든 사이비 종교 사건의 진실을 그린 '나는 신이다'의 두 번째 이야기로 대한민국을 충격에 빠뜨린 네 개의 참혹한 사건, 반복돼서는... 
---
제목: <b>넷플릭스</b> 8월 넷째 주 <b>신작</b> 추천
URL: https://www.bntnews.co.kr/article/view/bnt202508140174
내용: <b>넷플릭스</b>가 8월 넷째 주 신락 라인업을 발표했다. #나는 생존

In [27]:
news_qa("회사 가기 싫어.")

Prompt: 회사 가기 싫어.
---
News_Bot: Call Nothing
회사에 가기 싫은 날도 있죠. 혹시 특별한 이유가 있나요? 아니면 기분 전환을 위해 도움이 될 만한 정보를 찾아드릴까요?


# [심화] Multiple Tool Call
때로는, 하나의 메시지에서 여러 개의 Tool Call을 수행해야 하기도 합니다.   


반복문을 통해, Tool Call의 목록을 한꺼번에 전달합니다.

In [33]:
import json

def news_qa_v2(prompt, model='gpt-4o'):
    print('🟡 Prompt:', prompt)

    # 1. tool 이름에 맞는 실제 함수 매핑
    available_functions = {
        'web_search': get_news  # 반드시 tool name과 동일해야 함
    }

    # 2. LLM에 첫 메시지 전송하여 tool call 여부 확인
    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        }
    ], model=model)

    print('---')
    print('🟢 News_Bot: Call ', end='')

    # 3. tool 호출이 발생했을 경우
    if tool_call_result.tool_calls:
        tool_messages = []

        for tool_call in tool_call_result.tool_calls:
            name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            print(f"{name} {arguments}")

            if name not in available_functions:
                raise ValueError(f"알 수 없는 함수: {name}")

            # 툴 실행
            result = available_functions[name](**arguments)

            print('---')
            print('🟢 News_Bot:', end='')

            # Tool 응답 메시지 생성
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })

        # 4. tool 결과와 함께 최종 응답 생성
        final_response = news_bot_v2(
            [
                {
                    "role": "system",
                    "content": "사용자의 질문과 이를 답변하기 위한 정보가 tool 실행 결과를 통해 주어집니다. 전체 내용을 최대한 자세히 읽고, 상세하게 답변하세요."
                },
                {
                    "role": "user",
                    "content": prompt
                },
                tool_call_result,
                *tool_messages
            ],
            stream=True,
            model=model
        )

        for chunk in final_response:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                print(delta.content, end='')

    else:
        # tool 호출이 없으면 일반 대화로 응답
        print('🔵 Nothing')
        print(tool_call_result.content)


In [35]:
news_qa_v2(prompt, model='gpt-4o')

🟡 Prompt: 넷플릭스 신작 추천해줘.
---
🟢 News_Bot: Call web_search {'query': '넷플릭스 신작'}
제목: 다큐멘터리부터 SF 호러물까지⋯국내외 OTT <b>신작</b> 공개 [콘슐랭]
URL: https://n.news.naver.com/mnews/article/031/0000957584?sid=105
내용: 그날 그곳에 있었던 그들의 이야기 '나는 생존자다' <b>넷플릭스</b>는 15일 <b>신작</b>으로 '나는 생존자다'를 공개했다. 대한민국을 충격에 빠뜨린 네 개의 사건, 반복돼서는 안 될 이야기를 살아남은 사람들의 목소리로 기록한... 
---
제목: '나는 신이다' 후속작→'연애 실험', <b>넷플릭스 신작</b> '종합선물세트'
URL: https://sports.hankooki.com/news/articleView.html?idxno=6906955
내용: 8월 셋째 주, <b>넷플릭스</b>가 전 세계 시청자들의 시선을 사로잡을 <b>신작</b> 라인업을 공개했다. 참혹한 사건의 생존자 증언을 담은 다큐멘터리부터 숨 막히는 하룻밤 추격극, 얼굴 대신 마음으로만 이어지는 연애 실험, 인기... 
---
제목: [이번주 뭐보지] <b>넷플릭스</b> '나는 생존자다' 등 <b>신작</b> 공개
URL: https://www.newscj.com/news/articleView.html?idxno=3306067
내용: <b>넷플릭스</b>가 금주 <b>신작</b> 라인업을 15일 공개했다. '나는 생존자다'는 대한민국 사회를 뒤흔든 사이비 종교 사건의 진실을 그린 '나는 신이다'의 두 번째 이야기로 대한민국을 충격에 빠뜨린 네 개의 참혹한 사건, 반복돼서는... 
---
제목: <b>넷플릭스</b> 8월 넷째 주 <b>신작</b> 추천
URL: https://www.bntnews.co.kr/article/view/bnt202508140174
내용: <b>넷플릭스</b>가 8월 넷째 주 신락 라인업을 발표했다. #나